In [1]:
import pandas as pd
import numpy as np
# Visualizing pipelines in HTML
from sklearn import set_config; set_config()
# Exchange rates
from forex_python.converter import CurrencyRates
# NLP
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from sklearn.feature_extraction.text import CountVectorizer
# Suppress warning
pd.options.mode.chained_assignment = None

# Load Data

In [2]:
ev = pd.read_csv('../raw_data/enterprise_value_tidy.csv', index_col='Unnamed: 0')
df = pd.read_csv('../raw_data/company_info_fmp.csv', index_col='Unnamed: 0')
roic = pd.read_csv('../raw_data/ROIC.csv', index_col='Unnamed: 0')
financials = pd.read_csv('../raw_data/company_IS.csv', index_col='Unnamed: 0')

/home/harry/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Tidy 

In [3]:
financials = roic.merge(financials, on=['symbol', 'date'],how='inner')

## Feature Engineering

### Growth Rate

In [4]:
# Remove companies with fewer than 5 entries
symbol_counts = financials['symbol'].value_counts()
financials = financials[financials['symbol'].isin(symbol_counts[symbol_counts > 4].index)]

In [5]:
# Calculate growth rate
financials['old_revenue'] = financials['revenue'].shift(-4)

financials['growth_rate'] = (financials['revenue'] / financials['old_revenue'])**0.25 - 1

In [6]:
# Select 2019 entry
financials['date'] = pd.to_datetime(financials['date'])

financials = financials[financials['date'] < '2020-01-01']

financials = financials[financials['date'] >= '2019-01-01']

### Ebitda Margin

In [7]:
# Calculate ebitda margin
financials['ebitda_margin'] = financials['ebitda'] / financials['revenue']

In [8]:
financials.drop(columns='old_revenue', inplace=True)

## Filter Continuous Variables

### Company Size

In [9]:
# Remove small companies
financials = financials[financials['revenue'] > 10_000_000]
print(f'{len(financials)} rows remaining')

10066 rows remaining


### Absurd Values

In [10]:
# Remove companies with absurd growth
financials = financials[financials['growth_rate'] < 2]
print(f'{len(financials)} rows remaining')

9777 rows remaining


In [11]:
# Remove companies with negative ebitda margin
financials = financials[financials['ebitda_margin'] >= 0.01]
print(f'{len(financials)} rows remaining')

7355 rows remaining


In [12]:
# Merge tables
ev_info = ev.merge(df, on='symbol', how='inner')
ev_info = ev_info.merge(financials, on='symbol', how='inner')[['symbol', 'enterpriseValue', 'sector', 'currency',
                                                     'country', 'description', 'returnOnCapitalEmployed',
                                                     'revenue', 'grossProfit', 'ebitda', 'growth_rate', 'ebitda_margin']]

In [13]:
# Remove too large companies
ev_info = ev_info[ev_info['enterpriseValue'] < 200_000_000_000]
print(f'{len(ev_info)} rows remaining')

7099 rows remaining


In [14]:
# Remove companies with negative values
ev_info = ev_info[ev_info['enterpriseValue'] > 0]
print(f'{len(ev_info)} rows remaining')

5996 rows remaining


## Filter Categorical Variables

### Uncommon Values

In [15]:
# Keep most common countries
countries = ev_info['country'].value_counts()
ev_info = ev_info[ev_info['country'].isin(countries[countries > 20].index)]
print(f'{len(ev_info)} rows remaining')

5824 rows remaining


In [16]:
# Keep most common currencies
currencies = ev_info['currency'].value_counts()
ev_info = ev_info[ev_info['currency'].isin(currencies[currencies > 20].index)]
print(f'{len(ev_info)} rows remaining')

5811 rows remaining


### Sectors

In [17]:
# Keep most common sectors
sectors = ev_info['sector'].value_counts()
ev_info = ev_info[ev_info['sector'].isin(sectors[sectors >= 20].index)]
print(f'{len(ev_info)} rows remaining')

5609 rows remaining


In [18]:
# Remove financial/Real Estate companies
financial_sectors = ['Financial Services', 'Banking', 'Real Estate']
ev_info = ev_info[~ev_info['sector'].isin(financial_sectors)]
print(f'{len(ev_info)} rows remaining')

4813 rows remaining


In [19]:
# Drop companies with no sector or description
ev_info = ev_info.dropna(subset=['sector', 'description']).drop_duplicates()
print(f'{len(ev_info)} rows remaining')

4722 rows remaining


In [20]:
# Change GBp to GBP
ev_info = ev_info.replace({'GBp': 'GBP'})

In [21]:
sector_map = {'Health Care': 'Healthcare', 'Energy ': 'Energy', 'Building': 'Construction'}
ev_info['sector'] = ev_info['sector'].map(sector_map).fillna(ev_info['sector'])

### Missing Values

In [22]:
ev_info = ev_info.dropna()
print(f'{len(ev_info)} rows remaining')

4720 rows remaining


## Conversion

### Units

In [23]:
# Convert to millions
ev_info[['enterpriseValue', 'revenue', 'grossProfit', 'ebitda']] = ev_info[['enterpriseValue', 'revenue', 'grossProfit', 'ebitda']].apply(lambda x: x/1_000_000)

### Exchange Rates

In [24]:
def get_rate(foreign_currency, home_currency='USD'):
    c = CurrencyRates()
    rate = c.get_rate(foreign_currency, home_currency)
    return rate

In [25]:
ev_info['exchange_rate'] = ev_info['currency'].apply(get_rate)

In [26]:
columns = ['enterpriseValue', 'revenue', 'grossProfit', 'ebitda']
for column in columns:
    ev_info[column] =  ev_info[column] * ev_info['exchange_rate']

In [28]:
ev_info.drop(columns=['currency'], inplace=True)

In [29]:
ev_info.head()

,symbol,enterpriseValue,sector,country,description,returnOnCapitalEmployed,revenue,grossProfit,ebitda,growth_rate,ebitda_margin,exchange_rate
0,0001.HK,21680.263318,Industrials,HK,"CK Hutchison Holdings Limited, an investment h...",0.051154,38550.325378,21533.550310,12754.362536,0.157184,0.330850,0.128922
1,0002.HK,5156.227367,Utilities,HK,"CLP Holdings Limited, an investment holding co...",0.047146,10950.746563,3343.586867,2387.244959,0.023875,0.217998,0.128922
2,0003.HK,4078.093035,Utilities,HK,"The Hong Kong and China Gas Company Limited, t...",0.091021,5237.847758,2426.862628,1614.771322,0.082469,0.308289,0.128922
4,0006.HK,10.313744,Utilities,HK,"Power Assets Holdings Limited, an investment h...",0.080384,173.786586,173.786586,937.519325,0.007559,5.394659,0.128922
5,0007.HK,7.469858,Technology,HK,Hong Kong Finance Investment Holding Group Lim...,0.226738,69.360702,3.718491,127.456474,0.329895,1.837589,0.128922


# NLP Preprocessing

In [30]:
def clean_info(info):
    # Remove punctuation
    for p in string.punctuation:
        info = str(info).replace(p, '')   
    # Lower case
    info = info.lower()
    # Remove numbers
    info = ''.join(word for word in info if not word.isdigit())
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(info)
    info = [w for w in word_tokens if not w in stop_words]
    info = ' '.join(info)
    return info

In [33]:
def nlp_preprocessing(df, context=2, min_df=0.05):
    df_copy = df.copy()
    # Clean language columns
    df_copy['clean_info'] = df_copy['description'].apply(clean_info)
    # Vectorize
    vectorizer = CountVectorizer(ngram_range=(1,context), max_df=0.85, min_df=min_df)
    X = vectorizer.fit_transform(df_copy['clean_info'])
    # Convert back to df
    vect_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names(), index= df_copy.index)
    merged_df = df_copy.merge(vect_df, left_index=True, right_index=True, how='left')
    merged_df.drop(columns=['description','clean_info'], inplace=True)
    return merged_df

In [34]:
clean_df = nlp_preprocessing(ev_info)
clean_df.head()

,symbol,enterpriseValue,sector,country,returnOnCapitalEmployed,revenue,grossProfit,ebitda,growth_rate,ebitda_margin,...,united states,use,used,various,vehicles,water,well,wholesale,worldwide,worldwide company
0,0001.HK,21680.263318,Industrials,HK,0.051154,38550.325378,21533.550310,12754.362536,0.157184,0.330850,...,1,0,0,0,0,4,2,0,1,1
1,0002.HK,5156.227367,Utilities,HK,0.047146,10950.746563,3343.586867,2387.244959,0.023875,0.217998,...,0,0,0,0,0,0,0,0,0,0
2,0003.HK,4078.093035,Utilities,HK,0.091021,5237.847758,2426.862628,1614.771322,0.082469,0.308289,...,0,0,0,0,0,2,0,0,0,0
4,0006.HK,10.313744,Utilities,HK,0.080384,173.786586,173.786586,937.519325,0.007559,5.394659,...,0,0,0,0,0,0,1,0,0,0
5,0007.HK,7.469858,Technology,HK,0.226738,69.360702,3.718491,127.456474,0.329895,1.837589,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
#ev_info.to_csv('../raw_data/ev_info.csv')